# __Datenbereinigung Teil 2: Bereinigung der Anforderungsdaten__

In diesem Schritt werden die Daten bereinigt, welche für das Wörterbuch mit den häufigst genannten Anforderungen benötigt werden.

In [1]:
# Import der benötigten Libraries
from time import time
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup
from HanTa import HanoverTagger as ht
import string
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/00_all')

In [16]:
# Einlesen der Daten über ein csv File
data = pd.read_csv("Inserate_GEsamtdatenanalyse.csv", sep=";")
print(data.shape)
data.info()

(41924, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41924 entries, 0 to 41923
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       41924 non-null  int64 
 1   ad_id            41924 non-null  object
 2   active_start_on  41924 non-null  object
 3   active_end_on    41924 non-null  object
 4   title            41924 non-null  object
 5   text             41923 non-null  object
 6   year             41924 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


In [17]:
# Listenelemente in einer neuen Spalte abspeichern
start = time()

column = []
for i in data['text']:
    text=''
    soup = BeautifulSoup(i, 'lxml')
    tags = soup.find_all('li')
    #print(tags)
    for t in tags:
        text = text + ' ' + t.text 
    column.append(text)

data['Text_Anforderungen'] = column

print(f'Benötigte Zeit: {time() - start} Sekunden')

TypeError: object of type 'float' has no len()

In [4]:
# Lemmatisierung der Anforderungen
start = time()
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

for mail in data['Text_Anforderungen']:
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    index = data[data['Text_Anforderungen']==mail].index
    data.loc[index, 'Text_Anforderungen'] = (' '.join(lemma))
    
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 3253.0038492679596 Sekunden


In [5]:
# Text in Kleinbuchstaben umwandeln
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.lower()

In [6]:
# Satzzeichen löschen
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

data['Text_Anforderungen'] = data['Text_Anforderungen'].apply(lambda x: remove_punctuation(x))

In [7]:
# Tabs \t aus den Texten entfernen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("\t", "", regex=True)
data['Text_Anforderungen']

0        leitung des projekt sap nebenbuchhaltung fç¬r ...
1        abschluss in wirtschaftsinformatik oder gleich...
2        leiten und überwachen von it und organisations...
3        mehrjçïhrig und ausgewiesen erfahrung als it g...
4        verantwortlich fç¬r das unternehmensintern inf...
                               ...                        
25095    debitoren und kreditorenbuchhaltung verrechnun...
25096    erfassen und bewirtschaften von klientendaten ...
25097    führung der finanzbuchhaltung von zwei gesells...
25098    unterstützung debitoren  kreditoren unterstütz...
25099    koordination und begleitung der buchhaltungspr...
Name: Text_Anforderungen, Length: 25100, dtype: object

In [8]:
# Ähnliche Anforderungen manuell mit einem Hauptbegriff ersetzen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("teamplayer", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("verhandlungssicher", "verhandlungsgeschick", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kundenorientierte", "kundenorientierung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projekt management", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projektmanagement", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("zusammenarbeit", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikation", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikationsfähigkeiten", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dwh", "data warehouse", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dienstleistungsorientierung", "dienstleistungsorientiert", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("eigenverantwortung", "eigenverantwortlich", regex=True)

In [9]:
# Word-Tokenization 
#nltk.download('punkt')

start = time()
data['text_tok'] = data.apply(lambda row: nltk.word_tokenize(row['Text_Anforderungen']), axis=1)
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 12.778288125991821 Sekunden


In [10]:
# Stopwörter löschen
nltk.download('stopwords')

stop_words = set(stopwords.words('german'))
data['text_tok'] = data['text_tok'].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to C:\Users\Sven
[nltk_data]     Konermann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Tokenizing rückgängig machen, da die Tokenizierung nicht in eine csv-Datei überführt werden kann
column = []
for i in data['text_tok']:
    test = TreebankWordDetokenizer().detokenize(i)
    column.append(test)

data['Text_Anforderungen'] = column

In [12]:
# Löschen der tokenisierten Spalte
data = data.drop(['text_tok'], axis = 1)

In [13]:
# Erneutes Löschen der Duplikate, da sich die Texte verändert haben
data = data.drop_duplicates(subset='Text_Anforderungen', keep="last")

In [14]:
data.to_csv("Inserate_clean_Anforderungsanalyse.csv", sep=";")

nächster Schritt: Wörterbuch erstellen (Analyse_der_Anforderungen.ipynb)